In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import sentencepiece as spm
sp_model = spm.SentencePieceProcessor()
sp_model.Load('prepare/sp10m.cased.ms-en.model')

True

In [3]:
import tensorflow as tf
import tensorflow_text
import struct

unknown = b'\xff\xff\xff\xff'

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    for node in graph_def.node:
        
        if node.op == 'RefSwitch':
          node.op = 'Switch'
          for index in xrange(len(node.input)):
            if 'moving_' in node.input[index]:
              node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
          node.op = 'Sub'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'AssignAdd':
          node.op = 'Add'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'Assign':
          node.op = 'Identity'
          if 'use_locking' in node.attr: del node.attr['use_locking']
          if 'validate_shape' in node.attr: del node.attr['validate_shape']
          if len(node.input) == 2:
            node.input[0] = node.input[1]
            del node.input[1]
            
        if 'Reshape/shape' in node.name or 'Reshape_1/shape' in node.name:
            b = node.attr['value'].tensor.tensor_content
            arr_int = [int.from_bytes(b[i:i + 4], 'little') for i in range(0, len(b), 4)]
            if len(arr_int):
                arr_byte = [unknown] + [struct.pack('<i', i) for i in arr_int[1:]]
                arr_byte = b''.join(arr_byte)
                node.attr['value'].tensor.tensor_content = arr_byte
            
            if len(node.attr['value'].tensor.int_val):
                node.attr['value'].tensor.int_val[0] = -1
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [4]:
import json

with open('prepare/test-set-segmentation.json') as fopen:
    data = json.load(fopen)
    
X, Y = [], []
for x, y in data:
    X.append(x)
    Y.append(y)

In [5]:
g = load_graph('super-super-tiny-segmentation/frozen_model.pb')
x = g.get_tensor_by_name('import/inputs:0')
logits = g.get_tensor_by_name('import/SelectV2_3:0')
test_sess = tf.InteractiveSession(graph = g)

In [6]:
from tqdm import tqdm

batch_size = 10
results = []
for i in tqdm(range(0, len(X), batch_size)):
    batch_x = X[i: i + batch_size]
    batches = []
    for b in batch_x:
        batches.append(f'segmentasi: {b}')
    g = test_sess.run(logits, feed_dict = {x:batches})
    results.extend(g.tolist())

 40%|████      | 3992/9939 [1:47:18<48:48,  2.03it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 9939/9939 [2:46:55<00:00,  1.01s/it]  


In [7]:
results_Y = [sp_model.DecodeIds(r) for r in results]
results_Y[0], Y[0]

('Nihonbashi Mitsui Tower. Menara Nihonbashi Mitsui (Ri Ben Qiao San Jing tawa, "Nihonbashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo, Tokyo, Jepun. Dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. Pembinaannya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen Mitsukoshimae, Tokyo Metro.',
 'Nihonbashi Mitsui Tower. Menara Nihonbashi Mitsui (Ri Ben Qiao San Jing tawa, "Nihonbashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo, Tokyo, Jepun. Dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. Pembinaannya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen Mitsukoshimae, Tokyo Metro.')

In [8]:
def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [9]:
wer = []
for i in tqdm(range(len(results_Y))):
    wer.append(calculate_wer(Y[i], results_Y[i]))

100%|██████████| 99384/99384 [00:03<00:00, 31254.71it/s]


In [10]:
import numpy as np
np.mean(wer)

0.037882556485141314